**Company** : <br>
Tech Firm

**Notebook Function** : <br>
    This notebook generates the global reach measures
    
**Output File(s)** : <br>
    tech_diversity_bridging_all.csv.csv - The final output file containing the network measure

**Author(s)** : <br>
Lara Yang, Sarayu Anshuman

Install packages and import libraries

In [ ]:
import os
import sys
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np
from mittens import Mittens
import csv
from operator import itemgetter
import ujson as json
import re
from gensim.matutils import cossim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from statistics import mean 
from sklearn.decomposition import PCA
from collections import Counter
from datetime import timedelta
import multiprocessing
import random
from ast import literal_eval
import networkx.algorithms.community as nx_comm
from cdlib import algorithms

In [ ]:
#########################################################################
########### Helper Functions for Generating Mittens Embeddings ##########
#########################################################################
def _window_based_iterator(toks, window_size, weighting_function):
    for i, w in enumerate(toks):
        yield w, w, 1
        left = max([0, i-window_size])
        for x in range(left, i):
            yield w, toks[x],weighting_function(x)
        right = min([i+1+window_size, len(toks)])
        for x in range(i+1, right):
            yield w, toks[x], weighting_function(x)
    return

def glove2dict(glove_filename):
    """
    Reads word vectors into a dictionary
    Parameters
    ----------
    glove_filename : str
        Name of file that contains vectors
    Returns
    -------
    data : dict
        A dictionary matching words to their vectors
    """
    with open(glove_filename) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        data = {line[0]: np.array(list(map(float, line[1: ]))) for line in reader}
    return data

# Inspired by original build_weighted_matrix in utils.py in the Mittens paper source codebase
def build_weighted_matrix(emails,
        mincount=300, vocab_size=None, window_size=10,
        weighting_function=lambda x: 1 / (x + 1)):
    """
    Builds a count matrix based on a co-occurrence window of
    `window_size` elements before and `window_size` elements after the
    focal word, where the counts are weighted based on proximity to the
    focal word.
    Parameters
    ----------
    emails : list of dicts
        Emails converted from JSON formats
    mincount : int
        Only words with at least this many tokens will be included.
    vocab_size : int or None
        If this is an int above 0, then, the top `vocab_size` words
        by frequency are included in the matrix, and `mincount`
        is ignored.
    window_size : int
        Size of the window before and after. (So the total window size
        is 2 times this value, with the focal word at the center.)
    weighting_function : function from ints to floats
        How to weight counts based on distance. The default is 1/d
        where d is the distance in words.
    Returns
    -------
    X : pd.DataFrame
        Cooccurence matrix guaranteed to be symmetric because of the way the counts are collected.
    """
    wc = defaultdict(int)
    corpus = read_corpus(emails, sentence_delim=False)
    #---print("corpus------")
    #---print(corpus)
    if corpus is None:
        print("These emails are empty\t{}.\n".format(str(emails)))
        return pd.DataFrame()
    for toks in corpus:
        for tok in toks:
            wc[tok] += 1
    #---print("wc------")
    #---print(wc)
    if vocab_size:
        srt = sorted(wc.items(), key=itemgetter(1), reverse=True)
        vocab_set = {w for w, c in srt[: vocab_size]}
        #---print('using vocabsize')
    else:
        vocab_set = {w for w, c in wc.items() if c >= mincount} #this is being printed
        #---print('using mincount')
    vocab = sorted(vocab_set)
    n_words = len(vocab)
    #---print(f"n_words: {n_words}")
    # Weighted counts:
    counts = defaultdict(float)
    for toks in corpus:
        window_iter = _window_based_iterator(toks, window_size, weighting_function)
        for w, w_c, val in window_iter:
            if w in vocab_set and w_c in vocab_set:
                counts[(w, w_c)] += val
    X = np.zeros((n_words, n_words))
    for i, w1 in enumerate(vocab):
        for j, w2 in enumerate(vocab):
            X[i, j] = counts[(w1, w2)]
    X = pd.DataFrame(X, columns=vocab, index=pd.Index(vocab))
    return X

def read_corpus(emails, sentence_delim=False):
    """
    Parameters
    ----------
    emails : list of dict
        A list of emails converted from JSON formats
    sentence_delim : bool, optional
        If true, co-occurrences across sentence boundaries are ignored.
    Returns
    -------
    list of list of str
        Corpus converted from emails.
        If sentence_delim is false, returns a list of emails, which are represented as lists of tokens
        If sentence_delim is true, returns a list of sentences, which are represented as lists of tokens
    """
    if not sentence_delim:
        return [email['body'].replace('\n', ' ').replace("SENT_END", "").strip().split() for email in emails]
    else:
        return [sent.strip().split() for email in emails for line in email['body'].split('\n') for sent in line.split('SENT_END') if len(sent) > 0]

def output_embeddings(mittens_df, filename, compress=False):
    if compress:
        mittens_df.to_csv(filename + '.gz', quoting=csv.QUOTE_NONE, header=False, sep=" ", compression='gzip')
    else:
        mittens_df.to_csv(filename, quoting=csv.QUOTE_NONE, header=False, sep=" ")
    return

def isnull_wrapper(x):
    r = pd.isnull(x)
    if type(r) == bool:
        return r
    return r.any()

def cossim_with_none(vec1, vec2, vec_format='sparse'):
    """
    Auxiliary function that calls cossim function to test if vectors are None to prevent erroring out.
    Parameters
    ----------
    vec1 : list of (int, float), gensim sparse vector format
    vec2 : list of (int, float), gensim sparse vector format
    format : str, optional
        Either sparse or dense. If sparse, vec1 and vec2 are in gensim sparse vector format; use cossim function from gensim.
        Otherwise, vec1 and vec2 are numpy arrays and cosine similarity is hand calculated
    Returns
    -------
    float
        Cosine similarity between vec1 and vec2
    """
    if not (isnull_wrapper(vec1) or isnull_wrapper(vec2)):
        if vec_format == 'sparse':
            return cossim(vec1, vec2)
        elif vec_format == 'dense':
            if len(vec1) == 0 or len(vec2) == 0:
                return None
            return np.dot(vec1, vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))
        else:
            raise ValueError()
    return None

#########################################################################
############# Helper Functions for Working with JSON Emails #############
#########################################################################

def slice_user_corpus(emails, train_mode):
    """
    Similar to slice_user_corpus in the Coco, modified to work with tech firm data structure
    Parameters
    ----------
    emails : list of dict
        A list of emails converted from JSON formats to dictionaries 
    train_mode : str
        One of 'annual', 'quarterly', 'all'
        Indicates how to chunk up emails - into quarters, years, or both
    Returns
    -------
    timekey2emails : dict
        Matches quarters or years to respective emails
    """
    timekey2emails = defaultdict(list)
    for email in emails:
        if train_mode == 'annual':
            timekey2emails[email['year']].append(email)
        elif train_mode == 'quarterly':
            timekey2emails[email['quarter']].append(email)
        elif train_mode == 'halfyearly':
            timekey2emails[email['halfyear']].append(email)
        elif train_mode == 'thirdyearly': #1/3 or a year or 4 months
            timekey2emails[email['thirdyear']].append(email)
        elif train_mode == 'all':
            timekey2emails[email['year']].append(email)
            timekey2emails[email['quarter']].append(email)
    return timekey2emails

#########################################################################
############# Helper Functions for Working with Date Objects ############
#########################################################################
def str_to_datetime(date):
    if date is None:
        return None
    date = re.sub(r"(\([A-Z]{3,}\))", "", date)
    date = date.strip()
    dt = None
    for fmt in ('%a, %d %b %Y %H:%M:%S %z', '%d %b %Y %H:%M:%S %z', '%a %d, %b %Y %H:%M:%S %z', '%a, %d %b %Y %H:%M:%S %Z', '%a, %d %b %Y %H:%M:%S (%Z)', '%Y-%m-%d %H:%M:%S%z'):
        try:
            dt = datetime.strptime(date, fmt)
        except ValueError:
            pass
    return dt

def to_quarter(date, format):
    """
    Returns quarter of date as str using date formats used in tech firm.
    Support for string format is provided, but if both to_quarter and to_year are needed,
    it is computationally more efficient to first convert string to datetime and then call
    both to_quarter and to_year on the datetime object to avoid duplicated datetime conversion.
    """
    if date is None:
        return None
    year, month = 0, 0
    if format == 'str':
        dt = str_to_datetime(date)
        if dt is None:
            return None
        year = dt.year
        month = dt.month
    elif format == 'datetime':
        year = date.year
        month = date.month
    quarter = ((int(month)-1) // 3) + 1
    timekey = str(year) + 'Q' + str(quarter)
    return timekey

def to_year(date, date_type):
    """
    Returns year of date as str using date formats used in tech firm.
    Support for string format is provided, but if both to_quarter and to_year are needed,
    it is computationally more efficient to first convert string to datetime and then call
    both to_quarter and to_year on the datetime object to avoid duplicated datetime conversion.
    """
    if date is None:
        return None
    if date_type == 'str':
        date = re.sub(r"\([A-Z]{3,}\)", "", date)
        date = date.strip()
        dt = None
        for fmt in ('%a, %d %b %Y %H:%M:%S %z', '%d %b %Y %H:%M:%S %z', '%a %d, %b %Y %H:%M:%S %z', '%a, %d %b %Y %H:%M:%S %Z'):
            try:
                dt = datetime.strptime(date, fmt)
            except ValueError:
                pass
        if dt is None:
            return None
        return str(dt.year)
    elif date_type == 'datetime':
        return str(date.year)

def to_halfyear(date, format):
    """
    Returns quarter of date as str using date formats used in tech firm.
    Support for string format is provided, but if both to_quarter and to_year are needed,
    it is computationally more efficient to first convert string to datetime and then call
    both to_quarter and to_year on the datetime object to avoid duplicated datetime conversion.
    """
    if date is None:
        return None
    year, month = 0, 0
    if format == 'str':
        dt = str_to_datetime(date)
        if dt is None:
            return None
        year = dt.year
        month = dt.month
    elif format == 'datetime':
        year = date.year
        month = date.month
    halfyear = ((int(month)-1) // 6) + 1
    timekey = str(year) + 'HY' + str(halfyear)
    return timekey

def to_thirdyear(date, format):
    """
    Return third of year of date in string
    """
    year, month = 0, 0
    if date is None:
        return None
    if format == 'str':
        year = date[0:4]
        month = date[5:7]
    elif format == 'datetime':
        year = date.year
        month = date.month
    thirdyear = ((int(month)-1) // 4) + 1
    timekey = str(year) + 'TH' + str(thirdyear)
    return timekey

def word_similarity(model, w1, w2):
    """
    This is an auxilary function that allows for comparing one word to another word or multiple words
    If w1 and w2 are both single words, n_similarity returns their cosine similarity which is the same as 
    simply calling similarity(w1, w2)
    If w1 or w2 is a set of words, n_similarity essentially takes the mean of the set of words and then computes
    the cosine similarity between that vector mean and the other vector. This functionality is both reflected
    in its source code and has been verified manually.
    Parameters
    ----------
    model : KeyedVectors
        The model that contains all the words and vectors
    w1 : str or list
        The first word or word list to be compared
    w2 : str or list
        The second word or word list to be compared
    Returns
    -------
    float
        Cosine similarity between w1 and w2
    """
    if not isinstance(w1, list):
        w1 = [w1]
    if not isinstance(w2, list):
        w2 = [w2]
    w1 = [w for w in w1 if w in model.key_to_index]
    w2 = [w for w in w2 if w in model.key_to_index]
    if len(w1) == 0 or len(w2) == 0:
        return None
    return model.n_similarity(w1, w2)

#########################################################################
############## Helper Functions for Working with Dataframes #############
#########################################################################
def dict_to_df(index2rows, cols, index_name):
    """
    Parameters
    ----------
    index2rows : dict
        Dictionary mapping index to rows to be coverted
    cols : list
        List of column names of type str
    index : list
        List of index names
    Returns
    -------
    df : pd.DataFrame
        Constructed dataframe
    """
    if index2rows is None or len(index2rows) == 0:
        return None
    if len(index_name) == 1:
        df = pd.DataFrame.from_dict(index2rows, orient='index', columns=cols)
        df.index.name = index_name[0]
        df.sort_index(axis=0, inplace=True)
        return df
    else:
        df = pd.DataFrame.from_dict(index2rows, orient='index', columns=cols)
        df = pd.DataFrame(df, pd.MultiIndex.from_tuples(df.index, names=index_name))
        df.sort_index(axis=0, inplace=True)
        return df

def get_recipients(msg):
    """
    Return a set of recipients of the current message.
    self is removed from list of recipients if in recipients
    tech firm's from fields are all lists based on visual inspection
    but we check for the type of the field just in case
    """
    sender = msg['from'][0] if type(msg['from']) == list else msg['from']
    return set(msg.get('to', []) + msg.get('cc', []) + msg.get('bcc', [])) - set([sender])

def is_internal_msg(msg):
    recipients = get_recipients(msg)
    internal = True
    for r in recipients:
        if not re.match(r'\d+', r):
            internal = False
            break
    return internal
    
def extract_variables_from_file(file):
    """ 
    Extract relevant information from name of embedding file, with format: {}(_{})?.txt
    """
    file_chunks = file[0:-4].split('_')
    usr = file_chunks[0]
    time_key = file_chunks[1] if len(file_chunks) == 2 else None
    return (usr, time_key)

def month2timekey(month, time_key):
    """
    Converts month numbers to actual year or quarter
    """
    result = ''
    if time_key == 'year':
        year = 2006 + month // 12 
        if month % 12 > 2:
            year += 1
        result = str(year)
    elif time_key == 'quarter':
        year = 2006 + month // 12 
        remainder = month % 12
        if remainder > 2:
            year += 1
        quarter = ''
        if remainder <=2:
            quarter = 'Q4'
        elif remainder <= 5:
            quarter = 'Q1'
        elif remainder <= 8:
            quarter = 'Q2'
        else:
            quarter = 'Q3'
        result = str(year)+quarter
    return result

def extract_hr_df(hr_file, time_key=None):
    """
    Extract and preprocess tech HR data
    """
    hr = pd.read_csv(hr_file, index_col=0)
    # spell is a counter of the number of observations, not tenure
    hr['hire_month'] = hr.apply(lambda row : row['month'] if row['spell'] == 1 else np.nan, axis=1)
    hr['hire_month'] = hr['hire_month'].fillna(method='ffill')
    hr['tenure_months'] = hr.apply(lambda row : (row['month'] - row['hire_month'])+1, axis=1)
    hr['tenure_days'] = hr.apply(lambda row: row['tenure_months'] * 365/12, axis=1)
    hr['employeeid'] = hr['employeeid'].astype(str)
    hr['bonus_eligible'] = hr['bonus_eligible'].astype(int)
    if not time_key:
        hr.drop_duplicates(subset='employeeid', keep='last', inplace=True)
        hr = (hr[['employeeid', 'hire_month', 'sales', 'marketing', 'tech', 'vol_exit_event', 'invol_exit_event', 'manager', 'female', 'fav_rating', 'bonus', 'bonus_eligible', 'age', 'age2', 'cumbonus', 'tenure_months', 'tenure_days']])
        hr.rename(columns={"employeeid": "anon_id"}, inplace=True)
        hr.set_index('anon_id', inplace=True)
    else:
        hr[time_key] = hr['month'].apply(lambda row : month2timekey(row, time_key))
        hr['bonus'] = hr.groupby(['employeeid', time_key])['bonus'].transform('sum')
        hr['vol_exit_event'] = hr.groupby(['employeeid', time_key])['vol_exit_event'].transform('max')
        hr['invol_exit_event'] = hr.groupby(['employeeid', time_key])['invol_exit_event'].transform('max')
        hr['manager'] = hr.groupby(['employeeid', time_key])['manager'].transform('max')
        hr['fav_rating'] = hr.groupby(['employeeid', time_key])['fav_rating'].transform('max')
        hr['bonus_eligible'] = hr.groupby(['employeeid', time_key])['bonus_eligible'].transform('max')
        hr.drop_duplicates(subset=['employeeid', time_key], keep='last', inplace=True)
        hr = (hr[['employeeid', time_key, 'hire_month', 'sales', 'marketing', 'tech', 'vol_exit_event', 'invol_exit_event', 'manager', 'female', 'fav_rating', 'bonus', 'bonus_eligible', 'age', 'age2', 'cumbonus', 'tenure_months', 'tenure_days']])
        hr.rename(columns={"employeeid": "anon_id"}, inplace=True)
        hr.set_index(['anon_id', time_key], inplace=True)
    return hr

Print the current directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
num_cores = 9
home_dir = current_dir
corpus_dir = "/zfs/projects/faculty/amirgo-identification/identification-Sarayu/tech/training/email_data_v2"
output_dir = "/zfs/projects/faculty/amirgo-identification/identification-Sarayu/downsampled_network_files/tech"

test_dir = os.path.join(home_dir, "idtf_output_data_test")
hr_file = "/zfs/projects/faculty/amirgo-identification/identification-Sarayu/tech/cossim/tech_ph2_data_for_analysis.csv"

print(corpus_dir)
print(output_dir)
print(hr_file)

Now run functions to calculate the global reach measure, as well as other variations

In [ ]:
def find_all_neighbors(G_directed, u):
    """
    Find the union of in and out-neighbors of u in G_directed.
    """
    return set(G_directed.successors(u)).union(set(G_directed.predecessors(u)))

def generate_department_bridging(G_directed, usr2department, edges2weights, weighted):
    """
    Generate department-based EI index
    Parameter
    ---------
    G_directed : NetworkX graph
    usr2department : dict
    	Maps useres to departments
    edges2weights: dict
    	Maps directed weights to weights
    weighted : bool
    	Whether computations should be weighted
    Returns
    -------
    tuple
        A 2-tuple of dictionaries matching node (i.e., user ids) to department EI index using two kinds of neighbors.
    """
    node2embed, node2embed_all_neighbors = defaultdict(lambda : None), defaultdict(lambda : None)
    for u in G_directed:    
        if u in usr2department:
            within_department, without_department = 0, 0
            for v in G_directed.neighbors(u):
                if v in usr2department:
                    add = edges2weights[(u, v)] if weighted else 1
                    if usr2department[u] == usr2department[v]:
                        within_department += add
                    else:
                        without_department += add
            if (within_department + without_department == 0):
                node2embed[u] = np.nan
            else:
                node2embed[u] = ((without_department - within_department) / (without_department + within_department))
            
            within_department, without_department = 0, 0
            for v in find_all_neighbors(G_directed, u):
                if v in usr2department:
                    add = (edges2weights.get((u, v), 0) + edges2weights.get((v, u), 0)) if weighted else 1
                    if usr2department[u] == usr2department[v]:
                        within_department += add
                    else:
                        without_department += add
            if (within_department + without_department == 0):
                node2embed_all_neighbors[u] = np.nan
            else:
                node2embed_all_neighbors[u] = ((without_department - within_department) / (without_department + within_department))

    return (node2embed, node2embed_all_neighbors)

def generate_ego_bridging(G_directed, edges2weights, communities, weighted):
    """
    Generating community EI based on immediate alters. This function returns both the results of using successors only vs
    using both successors and predecessor nodes.
    The unweighted version of both successor only and successor and predecessor both measure have already been separately computed in
    tech_ego_bridging.py and tech_union_bridging.py. They are re-computed here so that there's a correctly computed weighted version
    of these measures.
    """
    node2embed, node2embed_all_neighbors = defaultdict(lambda : None), defaultdict(lambda : None)
    node2community = {node : i for i, c in enumerate(communities) for node in c}

    for u in G_directed:
        local_community = node2community[u]
        within_cluster, without_cluster = 0, 0
        for v in G_directed.neighbors(u):
            add = edges2weights[(u, v)] if weighted else 1
            if node2community[v] == local_community:
                within_cluster += add
            else:
                without_cluster += add

        if (without_cluster+within_cluster) == 0:
            node2embed[u] = np.nan
        else:
            node2embed[u] = (without_cluster-within_cluster)/(without_cluster+within_cluster)

        within_cluster, without_cluster = 0, 0
        for v in find_all_neighbors(G_directed, u):
            add = (edges2weights.get((u, v), 0) + edges2weights.get((v, u), 0)) if weighted else 1
            if node2community[v] == local_community:
                within_cluster += add
            else:
                without_cluster += add

        if (without_cluster+within_cluster) == 0:
            node2embed_all_neighbors[u] = np.nan
        else:
            node2embed_all_neighbors[u] = (without_cluster-within_cluster)/(without_cluster+within_cluster)
    
    return (node2embed, node2embed_all_neighbors)

def generate_global_bridging(G_directed, edges2weights, communities, weighted):
    """
    Generate the degree of global bridging of G_directed. This is basically the EI index of the number of alters' alters who are in ego's community to
    number of alters who are not in ego's community.
    Parameters
    ----------
    G : NetworkX graph
    weighted : bool
        Whether to engage in weighted computations
    node2community : dict
        Maps nodes to the communities they belong to
    Returns
    -------
    node2embed : dict
        Maps node to global bridging
    """
    node2embed, node2embed_all_neighbors = defaultdict(lambda : None), defaultdict(lambda : None)
    node2community = {node : i for i, c in enumerate(communities) for node in c}

    if weighted:
        for u in G_directed:
            local_community = node2community[u]
            within_cluster, without_cluster = 0, 0
            uv2weight = {v : edges2weights[(u, v)] for v in G_directed.neighbors(u)}
            total_weight = sum(uv2weight.values())
            for v in G_directed.neighbors(u):
                for w in G_directed.neighbors(v):
                    add = (uv2weight[v]/total_weight) * edges2weights[(v, w)]
                    if u == w:
                        continue
                    elif node2community[w] == local_community:
                        within_cluster += add
                    else:
                        without_cluster += add
            if (without_cluster+within_cluster) == 0:
                node2embed[u] = np.nan
            else:
                node2embed[u] = (without_cluster-within_cluster)/(without_cluster+within_cluster)

            all_neighbors = find_all_neighbors(G_directed, u)
            uv2weight = {v : (edges2weights.get((u, v), 0) + edges2weights.get((v, u), 0)) for v in all_neighbors}
            total_weight = sum(uv2weight.values())
            for v in all_neighbors:
                for w in find_all_neighbors(G_directed, v):
                    add = (uv2weight[v]/total_weight) * (edges2weights.get((v, w), 0) + edges2weights.get((w, v), 0))
                    if u == w:
                        continue
                    elif node2community[w] == local_community:
                        within_cluster += add
                    else:
                        without_cluster += add
            if (without_cluster+within_cluster) == 0:
                node2embed_all_neighbors[u] = np.nan
            else:
                node2embed_all_neighbors[u] = (without_cluster-within_cluster)/(without_cluster+within_cluster)
    else:
        for u in G_directed:
            local_community = node2community[u]
            within_cluster, without_cluster = 0, 0
            for v in G_directed.neighbors(u):
                for w in G_directed.neighbors(v):
                    if u == w:
                        continue
                    elif node2community[w] == local_community:
                        within_cluster += 1
                    else:
                        without_cluster += 1
            if (without_cluster+within_cluster) == 0:
                node2embed[u] = np.nan
            else:
                node2embed[u] = (without_cluster-within_cluster)/(without_cluster+within_cluster)

            within_cluster, without_cluster = 0, 0
            for v in find_all_neighbors(G_directed, u):
                for w in find_all_neighbors(G_directed, v):
                    if u == w:
                        continue
                    elif node2community[w] == local_community:
                        within_cluster += 1
                    else:
                        without_cluster += 1
            if (without_cluster+within_cluster) == 0:
                node2embed_all_neighbors[u] = np.nan
            else:
                node2embed_all_neighbors[u] = (without_cluster-within_cluster)/(without_cluster+within_cluster)
    return (node2embed, node2embed_all_neighbors)

def generate_community_diversity(G_directed, edges2weights, communities, weighted):
    """
    Computes the diversity of communities an individual is able to access in her local network. If weighted,
    each individual label is included as many times as the weight. When using bi-directional neighbors, 
    weights across both edges are summed up, with a non-existent edge treated as having zero weight.
    Parameters
    ----------
    G_directed : NetworkX DiGraph
    edges2weights : dict of {tuple : int}
    weighted : bool
    Returns
    ------
    tuple of (dict, dict, int)
    """
    node2community = {node : i for i, c in enumerate(communities) for node in c}
    node2herf, node2herf_all_neighbors = defaultdict(lambda : None), defaultdict(lambda : None)
    if weighted:
        for u in G_directed:
            alter_communities = [node2community[v] for v in G_directed.neighbors(u) for _ in range(edges2weights[(u, v)])]
            node2herf[u] = sum([(counts/len(alter_communities))**2 for group, counts in Counter(alter_communities).items()])
            alter_communities = [node2community[v] for v in find_all_neighbors(G_directed, u) for _ in range(
                edges2weights.get((u, v), 0) + edges2weights.get((v, u), 0))]
            node2herf_all_neighbors[u] = sum([(counts/len(alter_communities))**2 for group, counts in Counter(alter_communities).items()])
            
    else:
        for u in G_directed:
            alter_communities = [node2community[v] for v in G_directed.neighbors(u)]
            node2herf[u] = sum([(counts/len(alter_communities))**2 for group, counts in Counter(alter_communities).items()])
            
            alter_communities = [node2community[v] for v in find_all_neighbors(G_directed, u)]
            node2herf_all_neighbors[u] = sum([(counts/len(alter_communities))**2 for group, counts in Counter(alter_communities).items()])
    return (node2herf, node2herf_all_neighbors)

def generate_network_measures(timekey, edges, test_mode):
    """
    Generating network measures for a given time period using edges
    Parameters
    ----------
    timekey : str
        A string that represents the time period for which network measures are being computed
    edges : list
        A list of directd edges represented by 2-tuples
    test_mode : bool
        If true, restrict edges to a hundredth of all edges
    """
    
    if len(edges) < 10:
        sys.stderr.write('Returning empty network at %s with %d edges at %s.\n' % (timekey, len(edges), datetime.now()))
        return dict()

    G_directed = nx.DiGraph()
    sys.stderr.write('Generating weighted network measures for %s with %d edges at %s.\n' % (timekey, len(edges), datetime.now()))
    edges2weights = Counter(edges)
    weighted_edges = [(edge[0], edge[1], weight) for edge, weight in edges2weights.items()]
    G_directed.add_weighted_edges_from(weighted_edges)

    '''
    node_ids_in_graph = list(G_directed.nodes())
    node_ids_in_weights = list(weighted_edges.keys())
    missing_nodes = set(node_ids_in_weights) - set(node_ids_in_graph)
    for node_id in missing_nodes:
        weights.pop(node_id, None)
    print(f"length of missing nodes: {len(missing_nodes)}")
    '''
    
    usr_quarter2network_measures = defaultdict(list)

    sys.stderr.write('Computing community-based connectivity at %s.\n' % datetime.now())
    # pre-computing community membership so that the two methods share communities
    try:
        weighted_communities = algorithms.leiden(G_directed, weights='weight').communities
        unweighted_communities = algorithms.leiden(G_directed).communities

        node2ego_bridging_weighted, node2ego_bridging_all_neighbors_weighted = generate_ego_bridging(G_directed, edges2weights, weighted_communities, True)
        node2ego_bridging_unweighted, node2ego_bridging_all_neighbors_unweighted = generate_ego_bridging(G_directed, edges2weights, unweighted_communities, False)

        node2comm_bridging_weighted, node2comm_bridging_all_neighbors_weighted = generate_global_bridging(G_directed, edges2weights, weighted_communities, True)
        node2comm_bridging_unweighted, node2comm_bridging_all_neighbors_unweighted = generate_global_bridging(G_directed, edges2weights, unweighted_communities, False)

        node2comm_diversity_weighted, node2comm_diversity_all_neighbors_weighted = generate_community_diversity(G_directed, edges2weights, weighted_communities, True)
        node2comm_diversity_unweighted, node2comm_diversity_all_neighbors_unweighted = generate_community_diversity(G_directed, edges2weights, unweighted_communities, False)
    
        sys.stderr.write('Computing department-based bridging at %s.\n' % datetime.now())
        node2depart_bridging_weighted, node2depart_bridging_all_neighbors_weighted = generate_department_bridging(G_directed, quarter2usr2department[timekey], edges2weights, True)
        node2depart_bridging_unweighted, node2depart_bridging_all_neighbors_unweighted = generate_department_bridging(G_directed, quarter2usr2department[timekey], edges2weights, False)

        for n in G_directed:
            row = ([node2ego_bridging_unweighted[n], node2ego_bridging_all_neighbors_unweighted[n], node2comm_bridging_unweighted[n], node2comm_bridging_all_neighbors_unweighted[n], node2comm_diversity_unweighted[n], node2comm_diversity_all_neighbors_unweighted[n], len(unweighted_communities),
            node2ego_bridging_weighted[n], node2ego_bridging_all_neighbors_weighted[n], node2comm_bridging_weighted[n], node2comm_bridging_all_neighbors_weighted[n], node2comm_diversity_weighted[n], node2comm_diversity_all_neighbors_weighted[n], len(weighted_communities),
            node2depart_bridging_unweighted[n], node2depart_bridging_all_neighbors_unweighted[n], node2depart_bridging_weighted[n], node2depart_bridging_all_neighbors_weighted[n]])
            usr_quarter2network_measures[(n, timekey)] = row
        #print(f"len: {len(usr_quarter2network_measures)}")
    except: 
        print('error caused here')
        pass
    
    return dict(usr_quarter2network_measures)

def time_edges_to_df(time_edges, test_mode=False):
    """
    Calculates network measures using edge lists
    Parameters
    ----------
    time_edges : dict
        A dictionary that maps quarters (quarters only) to a list of edges belonging to that time period
    test_mode : bool, optional
        If true, only generate one network
    Returns
    -------
    df : pd.DataFrame
        A dataframe of network measures with user id and timekey_type as index
    """
    if test_mode:
        time_edges = {quarter:edges for quarter, edges in time_edges.items() if len(edges) > 5}
        test_timekey = random.choice(list(time_edges))
        sys.stderr.write("Testing timekey %s out of %d time periods.\n" % (test_timekey, len(time_edges)))
        network_measures = generate_network_measures(test_timekey, time_edges[test_timekey], test_mode)
    else:
        pool = multiprocessing.Pool(processes = num_cores)
        results = [pool.apply_async(generate_network_measures, args=(timekey, edges, test_mode, )) for timekey, edges in time_edges.items()]
        pool.close()
        pool.join()
        network_measures = defaultdict(list)
        for r in results:
            #print(r)
            #node_ids_from_r = r.get()
            #print(node_ids_from_r)
            network_measures.update(r.get())
    
    cols = (['ego_bridging_unweighted', 'ego_bridging_all_neighbors_unweighted', 'global_bridging_unweighted', 'global_bridging_all_neighbors_unweighted', 'comm_diversity_unweighted', 'comm_diversity_all_neighbors_unweighted', 'n_comm_unweighted',
    'ego_bridging_weighted', 'ego_bridging_all_neighbors_weighted', 'global_bridging_weighted', 'global_bridging_all_neighbors_weighted', 'comm_diversity_weighted', 'comm_diversity_all_neighbors_weighted', 'n_comm_weighted',
    'department_bridging_unweighted', 'department_bridging_all_neighbors_unweighted', 'department_bridging_weighted', 'department_bridging_all_neighbors_weighted'])
    
    df = dict_to_df(network_measures, cols, index_name=['anon_id', 'quarter'])
    return df.round(5)

def extract_quarterly_network_measures(corpus_dir, test_mode=False):
    """
    Main workhorse function for computing netwrork measures and writing them to file.
    Parameters
    ----------
    corpus_dir : str
        Location of cleaned emails
    test_mode : bool, optional
        If testing, modify file_name to include flags for testing in final output file name
    """
    if test_mode:
        global output_dir
        output_dir = test_dir
    
    edges_file = 'edges_corrected.txt'
    edges_file = os.path.join(output_dir, edges_file)
    
    quarterly_edges = defaultdict(list)
    sys.stderr.write("Reading edges from edge file at %s.\n" % str(datetime.now()))
    with open(edges_file, 'r') as f:
        for line in f:
            tup = literal_eval(line)
            quarterly_edges[tup[0]].append((tup[1], tup[2]))

    network_df_quarterly = time_edges_to_df(quarterly_edges, test_mode)
    network_quarterly_filename = os.path.join(output_dir, "tech_diversity_bridging_all.csv")
    network_df_quarterly.to_csv(network_quarterly_filename)
    return

In [ ]:
starttime = datetime.now()
test_mode = False
try:
    test_mode = sys.argv[1].lower() == 'test'
except IndexError as error:
    pass

hr_df_quarterly = extract_hr_df(hr_file, 'quarter')
hr_df_quarterly['department'] = np.select([(hr_df_quarterly['sales'] == 1), (hr_df_quarterly['tech'] == 1), (hr_df_quarterly['marketing'] == 1)], ['sales', 'tech', 'marketing'], default='other')
usr_quarter2department = hr_df_quarterly['department'].dropna().to_dict()
quarter2usr2department = defaultdict(dict)
for k, department in usr_quarter2department.items():
	u, quarter = k
	quarter2usr2department[quarter][u] = department

sys.stderr.write('Generating network measures at %s.\n' % datetime.now())
extract_quarterly_network_measures(corpus_dir, test_mode)

sys.stderr.write('Finished generating network measures at %s with a duration of %s.\n' % (datetime.now(), str(datetime.now()-starttime)))